In [ ]:
import csv
import itertools
import os
import pickle
import random
import re
from io import open
import math

import pandas as pd
import spacy
from spacy import displacy
from conllu import parse_incr, TokenList
#from spacy_conll import init_parser
#from spacy_conll.parser import ConllParser
from tqdm.notebook import tqdm
from tqdm.contrib.concurrent import process_map

In [ ]:
ud_hdt_sentences = {}
all_ud_hdt_sentences = []
faulty_sentences = {}
filepath = "../../libs/UD_German-HDT"
#nlp_conll = ConllParser(init_parser("de_core_news_sm", "spacy"))
for filename in tqdm(list(os.listdir("../../libs/UD_German-HDT"))):
    cur_sentences = []
    if filename.endswith('.conllu'):
        #for token_list in nlp_conll.parse_conll_file_as_spacy(os.path.join(filepath, filename)):
        #    cur_sentences.append(token_list)
        #ud_hdt_sentences[filename] = cur_sentences
        #all_ud_hdt_sentences += cur_sentences
            
        data_file = open(os.path.join(filepath, filename), "r", encoding="utf-8")
        for token_list in parse_incr(data_file):
            cur_sentences.append(token_list)
        ud_hdt_sentences[filename] = cur_sentences
        all_ud_hdt_sentences += cur_sentences
        
ud_hdt_sentences

In [ ]:
nlp = spacy.load("de_core_news_sm")

with open("../../data/pseudowords/annotations.csv", "r") as csv_file:
    data = [row for row in csv.DictReader(csv_file)]
data

In [ ]:
# In case some attributes are not pre-annotated, add them using spaCy:
completed_data = []
for incomplete_example in tqdm(data):
    example = incomplete_example
    nlp_example = nlp(example["text"])
    example["construction_id"] = int(example["construction_id"])
    example["pos_tags"] = []  # eval(example["pos_tags"])
    example["xpos_tags"] = []  # eval(example["xpos_tags"])
    example["dep_rels"] = []  # eval(example["dep_rels"])
    example["dep_heads"] = []  # eval(example["dep_heads"])
    example["kees"] = eval(example["kees"])
    if len(example["pos_tags"]) == 0:
        example["pos_tags"] = [str(token.pos_) for token in nlp_example]
    if len(example["xpos_tags"]) == 0:
        example["xpos_tags"] = [str(token.tag_) for token in nlp_example]
    if len(example["dep_rels"]) == 0:
        example["dep_rels"] = [str(token.dep_).upper() for token in nlp_example]
    if len(example["dep_heads"]) == 0:
        example["dep_heads"] = [str(token.head).lower() for token in nlp_example]
    completed_data.append(example)
data = completed_data

data

In [ ]:
# Group the dataset into a list of lists where the label of the dictionaries is identical:
data.sort(key=lambda example: int(example["construction_id"]))  # Grouping doesn't work without sorting first!
data = {constr: list(group) for constr, group in itertools.groupby(data, key=lambda example: example["construction_id"])}

len(data)

In [ ]:
list(data.keys())

In [ ]:
data

We now try to exclude one sentence after the other to see if we can fill the empty sets.

In [ ]:
def get_common_deps(constr, group):
    example_deps = []
    for example in group:
        # Variant with forms:
        tokens = [token for token in nlp(example["text"])]
        head_poss = [token.head.pos_ for token in nlp(example["text"])]
        head_tags = [token.head.tag_ for token in nlp(example["text"])]
        example_deps.append((example, tokens, head_poss, head_tags))
    
    num_excluded = len(group) - 2  # start with only two sentences which are left in the group
    common_deps = (set(), -1.0)
    
    while common_deps[1] != 0.0 and num_excluded > 0:
        last_common_deps = common_deps
        num_excluded -= 1
        exclude = itertools.combinations(range(len(group)), num_excluded)  # try to take away different sentences from the group
        
        # exclude sentences from the current group
        tries_left = 1000  # don't try for too long
        for exclude_list in exclude:
            if tries_left == 0:
                break
            else:
                tries_left -= 1
            smaller_deps = example_deps.copy()
            if num_excluded > 0:
                for e in sorted(list(exclude_list), reverse=True):  # sorting like this makes sure there are no index errors
                    try: 
                        smaller_deps.pop(e)
                    except:
                        print(exclude_list)
            group_deps = []
            for example, tokens, head_poss, head_tags in smaller_deps:
                group_deps.append([
                    (str(token).lower(), str(dep), str(head))       # token -dep-> token
                    for token, dep, head in zip(tokens, example["dep_rels"], example["dep_heads"])
                ] + [
                    (str(tag), str(dep), str(head))         # tag -dep-> token
                    for tag, dep, head in zip(example["xpos_tags"], example["dep_rels"], example["dep_heads"])
                ] + [
                    (str(pos), str(dep), str(head))         # pos -dep-> token
                    for pos, dep, head in zip(example["pos_tags"], example["dep_rels"], example["dep_heads"])
                ] + [
                    (str(token).lower(), str(dep), str(head_tag))   # token -dep-> tag
                    for token, dep, head_tag in zip(tokens, example["dep_rels"], head_tags)
                ] + [
                    (str(tag), str(dep), str(head_tag))     # tag -dep-> tag
                    for tag, dep, head_tag in zip(example["xpos_tags"], example["dep_rels"], head_tags)
                ] + [
                    (str(pos), str(dep), str(head_tag))     # pos -dep-> tag
                    for pos, dep, head_tag in zip(example["pos_tags"], example["dep_rels"], head_tags)
                ] + [
                    (str(token).lower(), str(dep), str(head_pos))   # token -dep-> pos
                    for token, dep, head_pos in zip(tokens, example["dep_rels"], head_poss)
                ] + [
                    (str(tag), str(dep), str(head_pos))     # tag -dep-> pos
                    for tag, dep, head_pos in zip(example["xpos_tags"], example["dep_rels"], head_poss)
                ] + [
                    (str(pos), str(dep), str(head_pos))     # pos -dep-> pos
                    for pos, dep, head_pos in zip(example["pos_tags"], example["dep_rels"], head_poss)
                ])
                
            common_deps = set.intersection(*map(set, group_deps))
            common_deps = (common_deps, len(smaller_deps) / len(group)) if len(common_deps) > 0 else (common_deps, 0.0)
        
            if common_deps[1] < last_common_deps[1] :
                common_deps = last_common_deps
                break  # found the best common set
                
    with open("../../out/matches/matches_filters.tsv", "a+") as file:
        file.write(str(constr) + "\t" + str(1.0 - common_deps[1]) + "\t" + str(common_deps[0]) + "\t" + str(smaller_deps) + "\n")
    
    return constr, common_deps

In [ ]:
with open("../../out/matches/matches_filters.tsv", "w") as file:
    file.write("constr\tfuzziness\tfilter\tsentences used\n")

common_deps = {}
for constr, group in tqdm(data.items(), desc="Construction"):
    # try finding a common depencency
    common_deps[constr] = get_common_deps(constr, group)

In [ ]:
common_deps = pd.read_csv("../../out/matches/matches_filters.tsv", sep="\t", header=0)
common_deps

We first filter for the existence of KE-LEX:

In [ ]:
filtered_sentences = {}
for constr in tqdm(common_deps["constr"]):
    filtered_sentences[constr] = []
    for i, sentence in tqdm(enumerate(all_ud_hdt_sentences), total=len(all_ud_hdt_sentences), leave=False):
        assert constr in data
        kees_lists = data[constr]
        # any list of ke-lex needs to match; but from this list, every ke-lex needs to match
        # sentence.split(), so subwords do not match!
        if any(all(keyword in [token["form"] for token in sentence] for keyword in entry['kees']) for entry in kees_lists):
            filtered_sentences[constr].append(sentence)
            
filtered_sentences

In [ ]:
with open("../../out/matches/filtered_sentences.pickle", "wb") as file:
    pickle.dump(filtered_sentences, file)

Now we can apply the filters we created before:

In [ ]:
with open("../../out/matches/matches_dep_constr.tsv", "w") as file:
    file.write("constr\tfuzziness (common dep)\tfuzziness (matches)\tsentence\n")

In [ ]:
scores = {}
for constr, deps, fuzziness in tqdm(zip(common_deps["constr"], common_deps["filter"], common_deps["fuzziness"]), total=len(common_deps)):
    example_outputs = 0
    deps = eval(deps)
    if set() in deps:
        continue
    for i, filtered_sentence in tqdm(enumerate(filtered_sentences[constr]), total=len(filtered_sentences[constr]), leave=False): # in ud_hdt_sentences["de_hdt-ud-test.conllu"]
        example_outputs += 1
        found = 0.0  # all group elements have to be found in the sentence, so let's check that in the end!
        for ex_token, ex_dep, ex_head in deps:
            for token in filtered_sentence:
                if not token["head"]:  # for some reason, there is no token["head"] sometimes...
                    continue
                deptoken = str(token["form"]).lower()
                deptoken_xpos = str(token["xpos"]).lower()
                deptoken_upos = str(token["upos"]).lower()
                deprel = str(token["deprel"]).upper()
                dephead = str(filtered_sentence[token["head"] - 1]["form"]).lower()
                dephead_xpos = filtered_sentence[token["head"] - 1]["xpos"]
                dephead_upos = filtered_sentence[token["head"] - 1]["upos"]
                    
                if (
                    ex_token in {deptoken, deptoken_xpos, deptoken_upos} and
                    ex_dep == deprel and
                    ex_head in {dephead, dephead_xpos, dephead_upos}
                ):
                    found += 1.0
        if found > 0.0:  # if any matches have been found
            if example_outputs <= 10:  # only print out 10 examples, otherwise this would take to much disk space
                with open(f"../../out/matches/graphics/{constr}_{i}.svg", "w") as out_file:
                    nlp_corpus_sentence = nlp(filtered_sentence.metadata["text"])
                    svg = displacy.render(nlp_corpus_sentence, style="dep", jupyter=False)
                    out_file.write(svg)
            # note the amount of fuzziness (how many examples have been used to create the common dependency set) and how many of them actually matched
            
            score = (1.0 - fuzziness, 1.0 - (found / float(len(deps))))
            with open("../../out/matches/matches_dep_constr.tsv", "a+") as file:
                file.write(str(constr) + "\t" + str(score[0]) + "\t" + str(score[1]) + "\t" + filtered_sentence.metadata["text"] + "\n")

In [ ]:
filters = pd.read_csv("../../out/matches/matches_filters.tsv", sep="\t")
filters

In [ ]:
matches = pd.read_csv("../../out/matches/matches_dep_constr.tsv", sep="\t")
matches